In [2]:
import requests
import time
import random
import polars as pl
from loguru import logger

def fetch_nse_data_api(symbol_list, fetch_date,):
    # 1. Setup Session with Human-like Headers
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0",
        "Accept": "*/*",
        "Accept-Language": "en-US,en;q=0.5",
    }
    session.headers.update(headers)

    # 2. "Warm up" the session by visiting the homepage first
    # This grabs the required cookies (bm_sv, etc.)
    try:
        session.get("https://www.nseindia.com", timeout=10)
    except Exception as e:
        logger.error(f"Failed to initialize session: {e}")
        return

    all_data = []
    
    for count, symbol in enumerate(symbol_list):
        if count % 20 == 0:
            logger.info(f"Progress: {count}/{len(symbol_list)}")
            # Optional: Clear/refresh session every 50 requests to stay "fresh"
        
        try:
            # The direct API endpoint
            api_url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"
            
            # 3. Request the data
            response = session.get(api_url, timeout=10)
            
            if response.status_code == 200:
                json_data = response.json()
                
                # Extracting specific fields from the NSE JSON structure
                industry_info = json_data.get('industryInfo', {})
                market_cap = json_data.get('marketDeptOrderBook', {}).get('tradeInfo', {}).get('totalMarketCap', 0)
                
                data = {
                    "timestamp": fetch_date,
                    "symbol": symbol,
                    "macro_economic_sector": industry_info.get('macro', '-'),
                    "sector": industry_info.get('sector', '-'),
                    "industry": industry_info.get('industry', '-'),
                    "basic_industry": industry_info.get('basicIndustry', '-'),
                    "market_cap_cr": market_cap
                }
                all_data.append(data)
            else:
                logger.warning(f"Failed for {symbol}: Status {response.status_code}")
                # Log to failed table if needed
            
            # 4. RANDOM DELAY to avoid cool-down
            # Using random variation prevents pattern detection
            time.sleep(random.uniform(1.5, 3.5))

        except Exception as e:
            logger.error(f"Error fetching {symbol}: {e}")
            time.sleep(10) # Wait longer if we hit an error

    # 5. Batch Write to Database (Much faster than row-by-row)
    if all_data:
        df = pl.DataFrame(all_data)
        return df

In [3]:
symbol_list = ["TATACAP"]
end_date = "2025-12-25"

In [4]:
df = fetch_nse_data_api(symbol_list=symbol_list, fetch_date=end_date)

2025-12-27 10:57:48.156 | INFO     | __main__:fetch_nse_data_api:29 - Progress: 0/1
2025-12-27 10:57:48.233 | WARNING  | __main__:fetch_nse_data_api:57 - Failed for TATACAP: Status 503
